In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
image = cv2.imread("ModulePictures/20UPGM23211132_PPPV2_43_AfterBonding_NOK.jpg")

In [50]:
col = 1000
state = False
count = 0

for i in range(image.shape[0]) :
    if image[i,col].sum() >= 695 :
        print(image[i,col])
        if not(state) :
            state = True
            count=count+1
    else :
        state = False
print(count)

[251 255 254]
[251 255 254]
[231 237 236]
[248 255 254]
[250 255 252]
[242 251 248]
[242 253 250]
[251 255 252]
[247 254 251]
[250 255 255]
[244 251 248]
[243 254 251]
[248 255 252]
[239 250 247]
[250 255 254]
[250 255 254]
[250 255 254]
[249 255 251]
[248 255 254]
[249 255 253]
[248 255 254]
[250 255 252]
[247 255 251]
[250 255 252]
[250 255 252]
[253 255 254]
[236 240 235]
[251 255 252]
[253 255 252]
[250 255 251]
[231 237 232]
[249 255 250]
[247 251 245]
[239 245 240]
[251 255 251]
[247 254 247]
[235 243 236]
[235 243 236]
[240 248 238]
[246 248 242]
[227 237 231]
[246 249 247]
[233 240 233]
[239 247 240]
[248 252 247]
[243 248 246]
[234 240 235]
[243 248 246]
[251 255 252]
[235 252 239]
[231 243 231]
[241 248 241]
[245 252 245]
[247 248 244]
[248 252 247]
[243 248 246]
[241 248 245]
[251 255 252]
[247 251 246]
[238 239 235]
[255 255 252]
[232 236 230]
[232 246 240]
[248 255 249]
[250 255 249]
[247 255 250]
[250 255 253]
[242 247 245]
[250 255 254]
[249 255 251]
[244 251 244]
[230 2

In [39]:
col = 800
state = False
count = 0

for i in range(1,image.shape[0]) :
    if image[i,col].sum() >= image[i-1,col].sum() :
        if not(state) :
            state = True
            count=count+1
    else :
        state = False
print(count)

895


In [59]:
col = 1000
state = False
count = 0

for i in range(image.shape[0]) :
    if image[i,col,1]/image[i,col].sum() <= 0.5 :
        print(image[i,col])
        if not(state) :
            state = True
            count=count+1
    else :
        state = False
print(count)

[146 149 147]
[150 153 151]
[151 157 152]
[150 154 148]
[137 143 138]
[107 113 108]
[76 80 74]
[73 79 74]
[86 92 87]
[108 114 109]
[139 143 138]
[149 155 150]
[151 156 154]
[148 152 147]
[143 149 144]
[135 140 138]
[128 134 129]
[132 138 133]
[145 151 146]
[153 160 153]
[145 151 146]
[109 116 109]
[67 74 67]
[54 61 54]
[54 61 54]
[67 74 67]
[84 92 82]
[112 120 110]
[135 142 135]
[145 151 146]
[157 164 157]
[154 162 155]
[147 154 147]
[133 140 133]
[127 134 127]
[106 113 106]
[83 90 85]
[77 85 78]
[77 86 76]
[82 90 83]
[121 130 120]
[157 165 158]
[164 172 162]
[154 161 154]
[143 149 144]
[133 139 134]
[129 135 130]
[127 133 128]
[134 140 135]
[143 150 143]
[141 148 141]
[122 130 120]
[75 82 75]
[47 54 47]
[35 41 36]
[36 43 36]
[52 60 53]
[ 97 104  97]
[128 135 128]
[141 149 142]
[153 160 153]
[156 165 155]
[158 167 157]
[157 165 158]
[153 161 154]
[136 143 136]
[108 115 108]
[89 97 90]
[82 90 83]
[80 87 80]
[90 97 90]
[123 130 123]
[155 162 155]
[166 173 166]
[155 162 155]
[134 141 134]